In [46]:
import pandas as pd
import pickle
import numpy as np

In [47]:
df = pickle.load(open('dataset_level2.pkl' , 'rb'))

In [48]:
df.head(5)

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,1,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,1,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,1,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,1,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,1,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground


In [49]:
df[df['city'].isnull()]['venue'].value_counts()

venue
Dubai International Cricket Stadium        3092
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sylhet International Cricket Stadium        128
Sharjah Cricket Stadium                     127
Carrara Oval                                 64
Name: count, dtype: int64

In [50]:
cities = np.where(df['city'].isnull(), df['venue'].str.split().apply(lambda x:x[0]), df['city'])

In [51]:
df['city'] = cities

In [52]:
df.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [53]:
df.drop(columns=['venue'],inplace = True)

In [54]:
df['city'].value_counts()

city
Colombo          4684
Dubai            4395
Johannesburg     3578
Mirpur           3190
Auckland         2845
                 ... 
Victoria          123
Nairobi           123
Rawalpindi        123
Potchefstroom     122
Cairns            122
Name: count, Length: 98, dtype: int64

In [55]:
eligible_cities = df['city'].value_counts()[df['city'].value_counts() > 600].index.tolist()

In [56]:
df = df[df['city'].isin(eligible_cities)]

In [57]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne
2,1,Australia,Sri Lanka,0.3,1,0,Melbourne
3,1,Australia,Sri Lanka,0.4,2,0,Melbourne
4,1,Australia,Sri Lanka,0.5,0,0,Melbourne
...,...,...,...,...,...,...,...
171868,1433,Sri Lanka,Australia,19.3,1,0,Colombo
171869,1433,Sri Lanka,Australia,19.4,0,0,Colombo
171870,1433,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo
171871,1433,Sri Lanka,Australia,19.6,2,0,Colombo


In [58]:
df['current_score'] = df.groupby('match_id')['runs'].cumsum()

In [59]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne,0
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne,0
2,1,Australia,Sri Lanka,0.3,1,0,Melbourne,1
3,1,Australia,Sri Lanka,0.4,2,0,Melbourne,3
4,1,Australia,Sri Lanka,0.5,0,0,Melbourne,3
...,...,...,...,...,...,...,...,...
171868,1433,Sri Lanka,Australia,19.3,1,0,Colombo,125
171869,1433,Sri Lanka,Australia,19.4,0,0,Colombo,125
171870,1433,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125
171871,1433,Sri Lanka,Australia,19.6,2,0,Colombo,127


In [60]:
df['over'] = df['ball'].apply(lambda x:str(x).split(".")[0])
df['ball_no'] = df['ball'].apply(lambda x:str(x).split(".")[1])

In [61]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2
2,1,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3
3,1,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4
4,1,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5
...,...,...,...,...,...,...,...,...,...,...
171868,1433,Sri Lanka,Australia,19.3,1,0,Colombo,125,19,3
171869,1433,Sri Lanka,Australia,19.4,0,0,Colombo,125,19,4
171870,1433,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,19,5
171871,1433,Sri Lanka,Australia,19.6,2,0,Colombo,127,19,6


In [62]:
df['balls_bowled'] = (df['over'].astype('int')*6) + df['ball_no'].astype('int')

In [63]:
df['balls_left'] = 120 - df['balls_bowled']
df['balls_left'] = df['balls_left'].apply(lambda x:0 if x<0 else x)

In [64]:
df['player_dismissed'] = df['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
df['player_dismissed'] = df['player_dismissed'].astype('int')
df['player_dismissed'] = df.groupby('match_id')['player_dismissed'].cumsum()
df['wickets_left'] = 10 - df['player_dismissed']

In [65]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119,10
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118,10
2,1,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117,10
3,1,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116,10
4,1,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
171868,1433,Sri Lanka,Australia,19.3,1,8,Colombo,125,19,3,117,3,2
171869,1433,Sri Lanka,Australia,19.4,0,8,Colombo,125,19,4,118,2,2
171870,1433,Sri Lanka,Australia,19.5,0,9,Colombo,125,19,5,119,1,1
171871,1433,Sri Lanka,Australia,19.6,2,9,Colombo,127,19,6,120,0,1


In [66]:
df['crr'] = (df['current_score']*6)/df['balls_bowled']

In [67]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119,10,0.000000
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118,10,0.000000
2,1,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117,10,2.000000
3,1,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116,10,4.500000
4,1,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115,10,3.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171868,1433,Sri Lanka,Australia,19.3,1,8,Colombo,125,19,3,117,3,2,6.410256
171869,1433,Sri Lanka,Australia,19.4,0,8,Colombo,125,19,4,118,2,2,6.355932
171870,1433,Sri Lanka,Australia,19.5,0,9,Colombo,125,19,5,119,1,1,6.302521
171871,1433,Sri Lanka,Australia,19.6,2,9,Colombo,127,19,6,120,0,1,6.350000


In [70]:
groups = df.groupby('match_id')

match_ids = df['match_id'].unique()
last_five = []

for id in match_ids:
    last_five.extend(
        groups.get_group(id)['runs']
        .rolling(window=30)
        .sum()
        .values
        .tolist()
    )

In [74]:
df['last_five'] = last_five

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_five
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119,10,0.000000,NaN
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118,10,0.000000,NaN
2,1,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117,10,2.000000,NaN
3,1,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116,10,4.500000,NaN
4,1,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115,10,3.600000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171868,1433,Sri Lanka,Australia,19.3,1,8,Colombo,125,19,3,117,3,2,6.410256,32.0
171869,1433,Sri Lanka,Australia,19.4,0,8,Colombo,125,19,4,118,2,2,6.355932,32.0
171870,1433,Sri Lanka,Australia,19.5,0,9,Colombo,125,19,5,119,1,1,6.302521,32.0
171871,1433,Sri Lanka,Australia,19.6,2,9,Colombo,127,19,6,120,0,1,6.350000,33.0


In [77]:
final_df = df.groupby('match_id').sum()['runs'].reset_index().merge(df, on='match_id')

In [79]:
final_df = final_df[['batting_team' , 'bowling_team' , 'city' ,'current_score' , 'balls_left' , 'wickets_left' , 'crr' , 'last_five' , 'runs_x']]

In [80]:
final_df.dropna(inplace = True)

In [82]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
runs_x           0
dtype: int64

In [84]:
final_df = final_df.sample(final_df.shape[0])

In [85]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
52242,South Africa,Australia,Centurion,36,82,8,5.684211,30.0,128
31385,India,England,Durban,154,21,8,9.333333,44.0,218
48187,Pakistan,Bangladesh,Mirpur,68,59,8,6.688525,23.0,135
1408,West Indies,Pakistan,Dubai,36,75,5,4.800000,21.0,115
30502,New Zealand,India,Johannesburg,137,24,5,8.562500,51.0,190
...,...,...,...,...,...,...,...,...,...
12062,New Zealand,India,Auckland,70,63,8,7.368421,33.0,132
59202,India,Australia,Melbourne,50,83,10,8.108108,41.0,184
46327,West Indies,England,Pallekele,154,12,6,8.555556,36.0,179
56615,South Africa,West Indies,Durban,88,70,10,10.560000,62.0,195


In [87]:
import pickle
pickle.dump(final_df,open('dataset_level3_feature_ready.pkl','wb'))